In [49]:
%reload_ext autoreload
%autoreload 2

import openpyxl
import pandas as pd
import numpy as np

from synthasizer.table import Table, Cell, detect
from synthasizer.transformations import *
from synthasizer.conditions import *
from synthasizer.heuristics import *
from synthasizer.error import ReconstructionError, ContentReconstructionError

In [19]:
sales = openpyxl.load_workbook("../data/icecream.xlsx")
sales = Table.from_openpyxl(sales['icecream']['A1':'G10'])
sales.color(0, 1)
sales.color(1, 1)
sales.color(2, 1)
sales.color(2, 0)
sales.color(6, 1)
display(sales)
display(sales.color_df)

           0        1     2     3       4      5       6
0       Type  Country  June  July  August  Total  Profit
1     Banana       BE   170   690     520   1380     YES
2                  DE   610   640     320   1570      NO
3                  DE   250   650     630   1530     YES
4  Chocolate       BE   560   320     140   1020     YES
5                  FR   430   350     300   1080     YES
6                                                       
7                  NL   210   280     270    760      NO
8  Speculaas       BE   300   270     290    860      NO
9    Vanilla       BE   610   190     670   1470     YES

,0,0,0,0,0,0,0
0,0,0,4,0,0,0,0
1,1,2,3,0,0,0,5
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0


In [22]:
sales_header = Header(1)(sales)
display(sales_header.df)
display(sales_header.color_df)

,Type,Country,June,July,August,Total,Profit
0,Banana,BE,170,690,520,1380,YES
1,,DE,610,640,320,1570,NO
2,,DE,250,650,630,1530,YES
3,Chocolate,BE,560,320,140,1020,YES
4,,FR,430,350,300,1080,YES
5,,,,,,,
6,,NL,210,280,270,760,NO
7,Speculaas,BE,300,270,290,860,NO
8,Vanilla,BE,610,190,670,1470,YES


,0,0,4,0,0,0,0
0,1,2,3,0,0,0,5
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0


In [13]:
AggregatedHeuristic(EmptyColumnHeuristic())(sales)

0.8428571428571429

In [12]:
AggregatedHeuristic(ValueColumnHeuristic())(sales)

0.8603174603174605

In [36]:
ColorRowHeuristic()(final)

1.0

In [42]:
heuristic = WeightedHeuristic(
    [
        EmptyHeuristic(),
        AggregatedHeuristic(ValueColumnHeuristic()),
        ColorRowHeuristic(),
    ],
)

In [37]:
heuristic(final)

<synthasizer.heuristics.EmptyHeuristic object at 0x000002A42029BC10> 1.0
AggregatedHeuristic(ValueColumnHeuristic()) 1.0
<synthasizer.heuristics.ColorRowHeuristic object at 0x000002A42029B7F0> 1.0


1.0

In [38]:
d2 = Delete(6, EmptyCondition())(
        Delete(4, StyleCondition('bold', True))(sales)
     )
d2

           0   1    2    3    4     5    6
0     Banana  BE  170  690  520  1380  YES
1             DE  610  640  320  1570   NO
2             DE  250  650  630  1530  YES
3  Chocolate  BE  560  320  140  1020  YES
4             FR  430  350  300  1080  YES
5             NL  210  280  270   760   NO
6  Speculaas  BE  300  270  290   860   NO
7    Vanilla  BE  610  190  670  1470  YES

In [42]:
error(d2)

1

In [27]:
final = Fold(2,5)(Delete(6, EmptyCondition())(Fill(0)(Header(1)(sales))))
final

         Type Country Profit variable value
0      Banana      BE    YES     June   170
1      Banana      DE     NO     June   610
2      Banana      DE    YES     June   250
3   Chocolate      BE    YES     June   560
4   Chocolate      FR    YES     June   430
5   Chocolate      NL     NO     June   210
6   Speculaas      BE     NO     June   300
7     Vanilla      BE    YES     June   610
8      Banana      BE    YES     July   690
9      Banana      DE     NO     July   640
10     Banana      DE    YES     July   650
11  Chocolate      BE    YES     July   320
12  Chocolate      FR    YES     July   350
13  Chocolate      NL     NO     July   280
14  Speculaas      BE     NO     July   270
15    Vanilla      BE    YES     July   190
16     Banana      BE    YES   August   520
17     Banana      DE     NO   August   320
18     Banana      DE    YES   August   630
19  Chocolate      BE    YES   August   140
20  Chocolate      FR    YES   August   300
21  Chocolate      NL     NO   A

In [28]:
ValueColumnHeuristic()(final)

array([1., 1., 1., 1., 1.])

## Nurse

In [52]:
nurse = detect(openpyxl.load_workbook("../data/nurse.xlsx")["Sheet1"])[0]

In [53]:
transformed = Stack(1, 29, 4)(Header(1)(nurse))
transformed.df

,,,M,A,N
0,,Mon,,,
1,Emma,,1,0,0
2,Lucas,,0,1,0
3,Noah,,0,0,1
4,Alice,,0,0,0
...,...,...,...,...,...
86,Ruben,,0,0,0
87,Ella,,0,0,0
88,Stan,,1,0,0
89,Nora,,0,1,0


In [54]:
EmptyHeuristic()(nurse)

0.7192118226600985

In [55]:
EmptyHeuristic()(transformed)

0.7538461538461538

In [56]:
# initialise heuristic
heuristic = WeightedHeuristic(
    [
        EmptyHeuristic(),
        AggregatedHeuristic(ValueColumnHeuristic()),
        ColorRowHeuristic(),
    ],
)

In [57]:
heuristic(nurse)

0.8878363016294051

In [58]:
heuristic(transformed)

0.9179487179487178

In [64]:
error = ContentReconstructionError()
error.initialise(nurse)
error(transformed)

Counter({Cell('M'): 6, Cell('A'): 6, Cell('N'): 6})
{Cell('Jules'), Cell('Noah'), Cell('Gabriel'), Cell('Lucas'), Cell('Stan'), Cell('Fri'), Cell('N'), Cell('Ruben'), Cell('Anna'), Cell('M'), Cell('Ella'), Cell(1), Cell('Thu'), Cell('Emma'), Cell('Tue'), Cell('Mon'), Cell('Jonas'), Cell('A'), Cell('Sat'), Cell(0), Cell('Wed'), Cell('Alice'), Cell('Nora'), Cell('Sun')}


0.0